In [10]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import sys
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
from datetime import datetime, timedelta

In [11]:
path="C:\\Users\\ninja\\Desktop\\6_proje_makalesi\\1_fall_detection\\proje dosyaları ve kodları\\veri_etiketleme_etl_pipeline\\veriler"
detections_path=path + "\\alp_sag_detection_dataDogru___20240821_175540_detections.csv"
imu_path=path +"\\alp_sag_imu_data_20240821_175456_imu_data.csv" # imu 


In [ ]:
class extract_fall_data():
    def __init__(self):
        try:
            pass
        except Exception as e :
            raise extract_fall_data(e,sys)
        
    def ekf_filter(self,z, x_est, P, F, Q, H, R):
    
        x_pred = np.dot(F, x_est)  # x_pred = F * x_est
        P_pred = np.dot(F, np.dot(P, F.T)) + Q  # P_pred = F * P * F.T + Q
    
    # Measurement update step
        y = z - np.dot(H, x_pred)  # Measurement residual
        S = np.dot(H, np.dot(P_pred, H.T)) + R  # Residual covariance
        K = np.dot(P_pred, np.dot(H.T, np.linalg.inv(S)))  # Kalman gain
    
        x_est_new = x_pred + np.dot(K, y)  # State update
        P_new = P_pred - np.dot(K, np.dot(H, P_pred))  # Covariance update
    
        return x_est_new, P_new
    
    def process_and_interpolate(self, filtered_df, desired_freq=50/1000):
        
        filtered_df['timestamp'] = pd.to_datetime(filtered_df['timestamp'], errors='coerce')
        filtered_df = filtered_df.dropna(subset=['timestamp'])
        filtered_df['timestamp_unix'] = (filtered_df['timestamp'].astype(np.int64) // 10**6)
        filtered_df = filtered_df.sort_values(by='timestamp')
        timestamps_unix = filtered_df['timestamp_unix'].values
        values = filtered_df[['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered']].values

        # Yeni zaman damgalarını oluşturma
        start_time = timestamps_unix.min()
        end_time = timestamps_unix.max()
        new_timestamps_unix = np.arange(start_time, end_time, 1 / desired_freq)

        # Cubic Spline interpolatör oluşturma
        cs = CubicSpline(timestamps_unix, values, axis=0, bc_type='natural')
        # Yeni zaman damgalarındaki verileri interpolasyon ile hesaplama
        new_values = cs(new_timestamps_unix)
        interpolated_df = pd.DataFrame(new_values, columns=['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered'])
        interpolated_df['timestamp_unix'] = new_timestamps_unix

        return interpolated_df
    
    def apply_threshold(self,interpolated_df):
        columns_to_zero = ['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered']

        for column in columns_to_zero:
            if column in interpolated_df.columns:  
                interpolated_df.loc[interpolated_df[column] > 20, column] = 0
                interpolated_df.loc[interpolated_df[column] < -20, column] = 0
            
        return interpolated_df
    
    def two_esp_extractor(self,detections_path,imu_path):
        detections_df=pd.read_csv(detections_path)
        imu_df=pd.read_csv(imu_path)
        imu_df_1 = imu_df[imu_df['topic'] == 'esp32_1/data']
        imu_df_2 = imu_df[imu_df['topic'] == 'esp32_2/data']
        imu_df_2 = imu_df_2.sort_values(by='timestamp')
        duplicate_timestamps = imu_df_2[imu_df_2.duplicated(subset='timestamp', keep=False)]
        duplicate_counts = duplicate_timestamps['timestamp'].value_counts()
        imu_df_2 = imu_df_2.drop_duplicates(subset='timestamp')
        imu_df_1 = imu_df_1.sort_values(by='timestamp')
        duplicate_timestamps = imu_df_1[imu_df_1.duplicated(subset='timestamp', keep=False)]
        duplicate_counts = duplicate_timestamps['timestamp'].value_counts()
        imu_df_1 = imu_df_1.drop_duplicates(subset='timestamp')
        F = np.eye(6)  
        H = np.eye(6)  
        Q = np.eye(6) * 0.01  
        R = np.eye(6) * 0.1  
        P = np.eye(6)  
        x_est = np.zeros(6)  

        
        filtered_values_1 = []
        filtered_values_2 = []

        
        for i in range(len(imu_df_1)):
            z = np.array([imu_df_1.iloc[i]['ax'], imu_df_1.iloc[i]['ay'], imu_df_1.iloc[i]['az'], 
                          imu_df_1.iloc[i]['gx'], imu_df_1.iloc[i]['gy'], imu_df_1.iloc[i]['gz']])
            x_est, P = self.ekf_filter(z, x_est, P, F, Q, H, R)
            filtered_values_1.append(x_est)

        
        for i in range(len(imu_df_2)):
            z = np.array([imu_df_2.iloc[i]['ax'], imu_df_2.iloc[i]['ay'], imu_df_2.iloc[i]['az'], 
                          imu_df_2.iloc[i]['gx'], imu_df_2.iloc[i]['gy'], imu_df_2.iloc[i]['gz']])
            x_est, P = self.ekf_filter(z, x_est, P, F, Q, H, R)
            filtered_values_2.append(x_est)

       
        filtered_df_1 = pd.DataFrame(filtered_values_1, columns=['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered'])
        filtered_df_1['timestamp'] = imu_df_1['timestamp'].reset_index(drop=True)

        filtered_df_2 = pd.DataFrame(filtered_values_2, columns=['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered'])
        filtered_df_2['timestamp'] = imu_df_2['timestamp'].reset_index(drop=True)

        filtered_df_1 = pd.DataFrame(filtered_values_1, columns=['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered'])
        filtered_df_1['timestamp'] = imu_df_1['timestamp'].reset_index(drop=True)

        filtered_df_2 = pd.DataFrame(filtered_values_2, columns=['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered'])
        filtered_df_2['timestamp'] = imu_df_2['timestamp'].reset_index(drop=True)


        interpolated_df_1 = self.process_and_interpolate(filtered_df_1)
        interpolated_df_2 = self.process_and_interpolate(filtered_df_2)

        interpolated_imu_1 = self.apply_threshold(interpolated_df_1)
        interpolated_imu_2 = self.apply_threshold(interpolated_df_2)


        return interpolated_imu_1,interpolated_imu_2,detections_df
    





# Tekra



In [30]:
extract=extract_fall_data()

In [34]:
df1,df2,detections_df=extract.two_esp_extractor(detections_path,imu_path)

In [42]:
df1

,ax_filtered,ay_filtered,az_filtered,gx_filtered,gy_filtered,gz_filtered,timestamp_unix
0,-1.028198,7.406667,-2.838919,-0.418559,0.063694,0.027297,1.724263e+12
1,-1.020539,7.627713,-2.904742,-0.430347,0.050697,0.045961,1.724263e+12
2,-1.013342,7.828984,-2.935427,-0.440230,0.041012,0.059841,1.724263e+12
3,-1.003664,7.984988,-2.912928,-0.446552,0.034748,0.066260,1.724263e+12
4,-0.980148,8.063047,-2.868723,-0.448961,0.024543,0.068392,1.724263e+12
...,...,...,...,...,...,...,...
10787,-5.037125,9.130744,2.043917,-0.420748,-0.557826,-0.418317,1.724263e+12
10788,-5.931060,9.493884,1.364587,-0.828691,-0.343283,-0.710905,1.724263e+12
10789,-5.822953,9.630155,0.399699,-1.072058,-0.113753,-0.879079,1.724263e+12
10790,-4.359680,9.429144,-0.768182,-1.042094,0.091744,-0.843613,1.724263e+12


In [36]:
df2

,ax_filtered,ay_filtered,az_filtered,gx_filtered,gy_filtered,gz_filtered,timestamp_unix
0,2.138565,8.406895,-2.373958,-0.012593,0.503110,-0.328804,1.724263e+12
1,2.661174,8.628310,-2.221580,0.042824,0.582171,-0.263299,1.724263e+12
2,2.965917,8.797170,-2.114343,0.075497,0.639309,-0.207668,1.724263e+12
3,3.073905,8.921844,-2.017026,0.090435,0.679164,-0.154188,1.724263e+12
4,3.135588,9.040206,-1.858517,0.106760,0.719177,-0.087611,1.724263e+12
...,...,...,...,...,...,...,...
8844,-5.026528,-5.062301,-10.102765,-1.571690,-1.652236,2.247382,1.724263e+12
8845,-5.152721,-4.135279,-10.261009,-1.458750,-1.378814,2.098301,1.724263e+12
8846,-5.882477,-2.932780,-11.091636,-1.340353,-1.018545,1.914958,1.724263e+12
8847,-6.300267,-2.165874,-11.474019,-1.237328,-0.747149,1.728567,1.724263e+12


In [38]:
detections_df

,Timestamp,Class Name,Confidence,Track ID
0,2024-08-21 17:55:44.876,falling,0.345433,NaN
1,2024-08-21 17:55:44.926,falling,0.479454,NaN
2,2024-08-21 17:55:44.976,falling,0.290418,NaN
3,2024-08-21 17:55:45.026,falling,0.515806,NaN
4,2024-08-21 17:55:45.126,falling,0.116733,NaN
...,...,...,...,...
3782,2024-08-21 17:59:06.176,not_falling,0.590795,3.0
3783,2024-08-21 17:59:06.226,not_falling,0.334215,3.0
3784,2024-08-21 17:59:06.276,not_falling,0.358066,3.0
3785,2024-08-21 17:59:06.326,not_falling,0.160431,3.0
